# **作业一  本地PaddleX图像分类示例**
![](https://ai-studio-static-online.cdn.bcebos.com/0b9c0844c3cb4845ab82606b4440eb27c50ea6643fdc44bda02fd27afd7bc621)
![](https://ai-studio-static-online.cdn.bcebos.com/8e396ee4c51d4f13adcfac4b6a1a76441ee67bee4b06448f8083b1ffae924101)
![](https://ai-studio-static-online.cdn.bcebos.com/bc290326a2934e3a96173222d5a3eb7597e027242751400fb716615d51dba9bb)
![](https://ai-studio-static-online.cdn.bcebos.com/a06bcd3e78584307a9863b862335e6fa170ab52e77ab4e33890bb7b00fa9d1b8)



# **作业二  泪河分割**

In [3]:
#解压数据集
!unzip -oq data/data81942/00.zip

In [ ]:
!pip install paddlex

Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 522kB 14.0MB/s eta 0:00:01
     |████████████████████████████████| 1.0MB 14.6MB/s eta 0:00:01
     |████████████████████████████████| 102kB 24.1MB/s ta 0:00:01
     |████████████████████████████████| 153kB 28.1MB/s eta 0:00:01
  Created wheel for pycocotools: filename=pycocotools-2.0.2-cp37-cp37m-linux_x86_64.whl size=278351 sha256=78fdd18bc03ae0ae9b4aae1206aa86e6d5eee3135a26100e612dab9da8781efc
  Stored in directory: /home/aistudio/.cache/pip/wheels/fb/44/67/8baa69040569b1edbd7776ec6f82c387663e724908aaa60963
Successfully built pycocotools


**#进行数据集的格式转换**

In [4]:

import paddlex as pdx
!paddlex --data_conversion --source labelme --to MSCOCO \
        --pics 00 \
        --annotations 00 \
        --save_dir dataset_coco

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/setuptools/depends.py:2: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp


**#进行数据集划分**

In [5]:

!paddlex --split_dataset --format COCO --dataset_dir dataset_coco --val_value 0.2 --test_value 0.1

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/setuptools/depends.py:2: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/colors.py:53: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is depre

**#定义图像处理流程transforms**

In [6]:

from paddlex.det import transforms
train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(), transforms.Normalize(),
    transforms.ResizeByShort(
        short_size=800, max_size=1333), transforms.Padding(coarsest_stride=32)
])

eval_transforms = transforms.Compose([
    transforms.Normalize(), transforms.ResizeByShort(
        short_size=800, max_size=1333), transforms.Padding(coarsest_stride=32)
])

**#定义数据集Dataset**

In [7]:

train_dataset = pdx.datasets.CocoDetection(
    data_dir='dataset_coco/JPEGImages',
    ann_file='dataset_coco/train.json',
    transforms=train_transforms,
    shuffle=True)
eval_dataset = pdx.datasets.CocoDetection(
    data_dir='dataset_coco/JPEGImages',
    ann_file='dataset_coco/val.json',
    transforms=eval_transforms)

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/colors.py:53: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sized


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
2021-04-21 16:03:40 [INFO]	Starting to read file list from dataset...
2021-04-21 16:03:40 [INFO]	293 samples in file dataset_coco/train.json
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
2021-04-21 16:03:40 [INFO]	Starting to read file list from dataset...
2021-04-21 16:03:40 [INFO]	83 samples in file dataset_coco/val.json


**#模型开始训练**

In [8]:

num_classes = len(train_dataset.labels) + 1

model = pdx.det.MaskRCNN(num_classes=num_classes, backbone='ResNet50')

model.train(
    num_epochs=50,
    train_dataset=train_dataset,
    train_batch_size=4,
    eval_dataset=eval_dataset,
    learning_rate=0.00125,
    warmup_steps=10,
    lr_decay_epochs=[8, 11],
    save_dir='output/mask_rcnn_r50_fpn',
    use_vdl=True)

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/math_op_patch.py:298: UserWarning: /opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddlex/cv/nets/detection/fpn.py:172
The behavior of expression A + B has been unified with elementwise_add(X, Y, axis=-1) from Paddle 2.0. If your code works well in the older versions but crashes in this version, try to use elementwise_add(X, Y, axis=0) instead of A + B. This transitional warning will be dropped in the future.
  op_type, op_type, EXPRESSION_MAP[method_name]))
2021-04-21 16:03:59,191 - INFO - If regularizer of a Parameter has been set by 'fluid.ParamAttr' or 'fluid.WeightNormParamAttr' already. The Regularization[L2Decay, regularization_coeff=0.000100] in Optimizer will not take effect, and it will only be applied to other Parameters!
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/math_op_patch.py:298: UserWarning: /opt/conda/envs/python35-pad

2021-04-21 16:04:00 [INFO]	Downloading ResNet50_cos_pretrained.tar from https://paddle-imagenet-models-name.bj.bcebos.com/ResNet50_cos_pretrained.tar


100%|██████████| 100310/100310 [00:19<00:00, 5185.29KB/s] 


2021-04-21 16:04:20 [INFO]	Decompressing output/mask_rcnn_r50_fpn/pretrain/ResNet50_cos_pretrained.tar...
2021-04-21 16:04:23 [INFO]	Load pretrain weights from output/mask_rcnn_r50_fpn/pretrain/ResNet50_cos_pretrained.
2021-04-21 16:04:24 [INFO]	There are 265 varaibles in output/mask_rcnn_r50_fpn/pretrain/ResNet50_cos_pretrained are loaded.
2021-04-21 16:04:31 [INFO]	[TRAIN] Epoch=1/50, Step=2/73, loss=2.776481, loss_cls=0.730708, loss_bbox=0.0, loss_mask=0.926402, loss_rpn_cls=0.686705, loss_rpn_bbox=0.432666, lr=0.0005, time_each_step=3.79s, eta=4:56:34
2021-04-21 16:04:32 [INFO]	[TRAIN] Epoch=1/50, Step=4/73, loss=1.840181, loss_cls=0.491164, loss_bbox=0.0, loss_mask=0.558688, loss_rpn_cls=0.684736, loss_rpn_bbox=0.105593, lr=0.000667, time_each_step=2.14s, eta=2:47:50
2021-04-21 16:04:33 [INFO]	[TRAIN] Epoch=1/50, Step=6/73, loss=1.70464, loss_cls=0.290104, loss_bbox=0.0, loss_mask=0.539775, loss_rpn_cls=0.68382, loss_rpn_bbox=0.190941, lr=0.000833, time_each_step=1.61s, eta=2:6:11

100%|██████████| 83/83 [00:17<00:00,  4.67it/s]


...

creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.05s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.327
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.806
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.159
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.327
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.365
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.504
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.504
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0

100%|██████████| 83/83 [00:14<00:00,  5.73it/s]


creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.05s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.321
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.788
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.148
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.321
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.354
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.494
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.494
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0

100%|██████████| 83/83 [00:16<00:00,  4.96it/s]


creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.05s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.324
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.791
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.151
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.324
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.360
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.494
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.494
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0

**#模型预测**

In [10]:

model = pdx.load_model('output/mask_rcnn_r50_fpn/best_model')
image_name = 'dataset_coco/JPEGImages/lhgd227.jpg'
result = model.predict(image_name)
pdx.det.visualize(image_name, result, threshold=0.5,save_dir="output/mask_rcnn_r50_fpn")

2021-04-21 17:04:31 [INFO]	Model[MaskRCNN] loaded.
2021-04-21 17:04:31 [INFO]	The visualized result is saved as output/mask_rcnn_r50_fpn/visualize_lhgd227.jpg


**#模型导出**

In [12]:

!paddlex --export_inference --model_dir='output/mask_rcnn_r50_fpn/best_model' --save_dir='output/best_model'

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/setuptools/depends.py:2: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/math_op_patch.py:298: UserWarning: /opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddlex/cv/nets/detection/fpn.py:172
The behavior of expression A + B has been unified with elementwise_add(X, Y, axis=-1) from Paddle 2.0. If your code works well in the older versions but crashes in this version, try to use elementwise_add(X, Y, axis=0) instead of A + B. This transitional warning will be dropped in the future.
  op_type, op_type, EXPRESSION_MAP[method_name]))
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/math_op_patch.py:298: UserWarning: /opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddlex/cv/

**预测结果**
![](https://ai-studio-static-online.cdn.bcebos.com/4c545f36c8c04d6d9f72b447591066679dd5881133c7453dabef41bcc0b86ae0)
![](https://ai-studio-static-online.cdn.bcebos.com/3088be60f3224b51b46f74586d6533d7c63761fea2da49cf8881b978fd6a3056)



请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 